# Predict Taxi Trip Duration

In [1]:
# Necessary import
import pickle
import mlflow # for experiment tracking
import pandas as pd
import xgboost as xgb

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
from pathlib import Path

# Python version
!python -V

Python 3.9.18


In [2]:
# Set the tracking uri
mlflow.set_tracking_uri("http://127.0.0.1:5000") # where metadata are stored
# Set the experiment (creating it if doesn't exist)
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1749675070001, experiment_id='1', last_update_time=1749675070001, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# Function for wrangling the data
def read_dataframe(filename):
    # If the data is a csv file
    if filename.endswith('.csv'):
        # Read csv
        df = pd.read_csv(filename)
        # Convert to datetime
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    # else if it is a parquet file
    elif filename.endswith('.parquet'):
        # read parquet
        df = pd.read_parquet(filename)
    # Not considered file type
    else:
        print("File not supported") # error message

    # Feature Engineering
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime # target var
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) # for minutes
    # Filtering durations
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Categorical features selection
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str) # convert integers to string

    # Feature Engineering to combine taxi trips origine and destination
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    # return the dataframe
    return df

In [4]:
# Read the train and validation sets
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet') # train
# using the url: df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet') 
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet') # validation
# using the url: df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

# Data size
len(df_train), len(df_val)

(73908, 61921)

In [5]:
# Set feature variables
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID
numerical = ['trip_distance']

In [6]:
# Initialize one-hot encoder
dv = DictVectorizer()

# Train data dictionaries
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
# Encoder training
X_train = dv.fit_transform(train_dicts)

# Validation data dictionaries
val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
# Validation data encoding
X_val = dv.transform(val_dicts)

In [7]:
# Set target variables
target = 'duration'
# Train and validation target vectors
y_train = df_train[target].values
y_val = df_val[target].values

## Saving an XG-Boost model With MLFlow

In [8]:
# Folder path for saving models
models_folder = Path('models')
# Create the models folder
models_folder.mkdir(exist_ok = True)

In [9]:
# Disable autolog
mlflow.xgboost.autolog(disable = True) # You can use autolog to automatically  log some information

In [10]:
# Start a new experiment run
with mlflow.start_run():

    # Prepare the data with XG-Boost special format
    train = xgb.DMatrix(X_train, label = y_train) # train
    valid = xgb.DMatrix(X_val, label = y_val) # val

    # XG-Boost Model optimal parameters
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    # Log the model parameters
    mlflow.log_params(best_params) # many parameters at once

    # Model training
    booster = xgb.train(
        params = best_params,
        dtrain = train,
        num_boost_round = 30,
        evals = [(valid, 'validation')],
        early_stopping_rounds = 50
    )

    # Make predictions
    y_pred = booster.predict(valid)
    # Compute the RMSE
    rmse = root_mean_squared_error(y_val, y_pred)
    # Log the model metric
    mlflow.log_metric("rmse", rmse)

    # Write a new binary file in the models folder
    with open("models/preprocessor.b", "wb") as f_out:
        # Save the encoder to that file
        pickle.dump(dv, f_out)
    # Log the encoder file as preprocessor
    mlflow.log_artifact("models/preprocessor.b", artifact_path = "preprocessor")

    # Log the XG-Boost model
    mlflow.xgboost.log_model(booster, artifact_path = "models_mlflow")    

/opt/anaconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:00:09] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773


/opt/anaconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:00:14] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/06/11 23:00:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run classy-goat-254 at: http://127.0.0.1:5000/#/experiments/1/runs/57937787c7a14831aa88b8982e5d16ce
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


---